NLP

IMDB sentiment classification (10.10.2024)

TEAM: Dutt Salveen Singh, Plichta Weronika, Tiurina Karina

In [1]:
!pip install transformers datasets gensim fasttext tensorflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.4/73.4 kB 3.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached pybind11-2.13.6-py3-none-any.whl.metadata (9.5 kB)
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.0 MB/s eta 0:00:00
Using cached pybind11-2.13.6-py3-none-any.whl (243 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.1 MB/s eta 0:00:00
  Created wheel for fasttext: filename=fasttext-0.9.3-cp310-cp310-linux_x86_64.whl size=4296189 sha256=e1090321097f21cee5eec6d14c9dd12586458446fb29c14771e0a93ee58c4faa
  Stored in directory: /r

In [2]:
from datasets import load_dataset
imdb = load_dataset("imdb")

import pandas as pd

results_df = pd.DataFrame(columns=['model', 'F1', 'recall', 'precision'])

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.81k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

## BERT

In [ ]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from datasets import Dataset, load_dataset
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import numpy as np

# Load the IMDB dataset
imdb = load_dataset("imdb")

train_dataset = imdb['train'].shuffle(seed=42).select(range(int(len(imdb['train']) * 0.1)))
test_dataset = imdb['test'].shuffle(seed=42).select(range(int(len(imdb['test']) * 0.1)))


# Load pre-trained BERT tokenizer and model
model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)  # Binary classification (positive/negative)

# Prepare data for BERT
def preprocess_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

# Apply preprocessing to the datasets
train_dataset = train_dataset.map(preprocess_function, batched=True)
test_dataset = test_dataset.map(preprocess_function, batched=True)

# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

# Define compute_metrics function
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    f1 = f1_score(labels, preds, average='weighted')
    precision = precision_score(labels, preds, average='weighted')
    recall = recall_score(labels, preds, average='weighted')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

# Create Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics
)

# Train the model
trainer.train()

# Evaluate on the test set
eval_results = trainer.evaluate()

# Extract metrics
f1_bert = eval_results['eval_f1']
recall_bert = eval_results['eval_recall']
precision_bert = eval_results['eval_precision']

# Create a new row for BERT results
# Assuming you have a results_df DataFrame already defined to store results
# If not, create an empty one before this line: results_df = pd.DataFrame(columns=['model', 'F1', 'recall', 'precision'])
new_row = pd.DataFrame({'model': ['BERT'], 'F1': [f1_bert], 'recall': [recall_bert], 'precision': [precision_bert]})

# Add the new row to the results DataFrame
try:
    results_df = pd.concat([results_df, new_row], ignore_index=True)  # If results_df exists
except NameError:
    results_df = new_row  # If results_df doesn't exist

print(results_df)

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

...

Map:   0%|          | 0/2500 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,No log,0.399050,0.806000,0.799135,0.852500,0.806000
2,0.364700,0.428872,0.906000,0.905935,0.906882,0.906000
3,0.364700,0.420054,0.912800,0.912779,0.913058,0.912800


  model        F1  recall  precision
0  BERT  0.799135   0.806     0.8525


<ipython-input-2-77248d320f9c>:80: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results_df = pd.concat([results_df, new_row], ignore_index=True)  # If results_df exists


In [3]:
# Manually adding the result above to avoid training BERT again after kernel restart
new_row = pd.DataFrame({'model': ['BERT'], 'F1': [0.799135], 'recall': [0.806], 'precision': [0.8525]})
results_df = pd.concat([results_df, new_row], ignore_index=True)  # If results_df exists

<ipython-input-3-3875f505d4c4>:3: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results_df = pd.concat([results_df, new_row], ignore_index=True)  # If results_df exists


## CountVectorizer + LR (reference to compare with other vectorizers)

In [4]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from datasets import Dataset, load_dataset
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import numpy as np
import gensim.downloader as api
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score, confusion_matrix
from sklearn.feature_extraction.text import CountVectorizer

# Load the IMDB dataset
imdb = load_dataset("imdb")

train_dataset = imdb['train'].shuffle(seed=42).select(range(int(len(imdb['train']) * 0.1)))
test_dataset = imdb['test'].shuffle(seed=42).select(range(int(len(imdb['test']) * 0.1)))

y_train = np.array(train_dataset['label'])
y_test = np.array(test_dataset['label'])

vectorizer = CountVectorizer(stop_words='english')
X_train = vectorizer.fit_transform(train_dataset['text'])
X_test = vectorizer.transform(test_dataset['text'])

model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
precision = precision_score(y_test, y_pred, average='weighted')
cm = confusion_matrix(y_test, y_pred)

print("Accuracy:", accuracy)
print("F1 Score:", f1)
print("Recall:", recall)
print("Precision:", precision)
print("Confusion Matrix:\n", cm)

new_row = pd.DataFrame({'model': ['CountVectorizer'], 'F1': [f1], 'recall': [recall], 'precision': [precision]})
results_df = pd.concat([results_df, new_row], ignore_index=True)  # If results_df exists

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Accuracy: 0.8368
F1 Score: 0.8366123661867848
Recall: 0.8368
Precision: 0.8380602170034239
Confusion Matrix:
 [[1000  243]
 [ 165 1092]]


## Word2Vec + LR

In [5]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from datasets import Dataset, load_dataset
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import numpy as np
import gensim.downloader as api
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score, confusion_matrix

# Load the IMDB dataset
imdb = load_dataset("imdb")

train_dataset = imdb['train'].shuffle(seed=42).select(range(int(len(imdb['train']) * 0.1)))
test_dataset = imdb['test'].shuffle(seed=42).select(range(int(len(imdb['test']) * 0.1)))

y_train = np.array(train_dataset['label'])
y_test = np.array(test_dataset['label'])

# Load pre-trained Word2Vec model
word2vec_model = api.load("word2vec-google-news-300")  # Google Word2Vec model with 300 dimensions

import numpy as np

def get_average_word2vec(text, model, vector_size):
    words = word_tokenize(text.lower())  # Tokenize the review
    word_vectors = [model[word] for word in words if word in model]
    if len(word_vectors) > 0:
        return np.mean(word_vectors, axis=0)  # Average the word vectors
    else:
        return np.zeros(vector_size)  # Return a vector of zeros if no words are in the model

# Apply this function to the train and test datasets
X_train = np.array([get_average_word2vec(text, word2vec_model, 300) for text in train_dataset['text']])
X_test = np.array([get_average_word2vec(text, word2vec_model, 300) for text in test_dataset['text']])

model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
precision = precision_score(y_test, y_pred, average='weighted')
cm = confusion_matrix(y_test, y_pred)

print("Accuracy:", accuracy)
print("F1 Score:", f1)
print("Recall:", recall)
print("Precision:", precision)
print("Confusion Matrix:\n", cm)

new_row = pd.DataFrame({'model': ['Word2Vec'], 'F1': [f1], 'recall': [recall], 'precision': [precision]})
results_df = pd.concat([results_df, new_row], ignore_index=True)  # If results_df exists

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


[==================================================] 100.0% 1662.8/1662.8MB downloaded
Accuracy: 0.798
F1 Score: 0.7979995151825465
Recall: 0.798
Precision: 0.7979992857307435
Confusion Matrix:
 [[ 990  253]
 [ 252 1005]]


## Word2Vec + LSTM

In [7]:
import numpy as np
import gensim.downloader as api
import nltk
from nltk.tokenize import word_tokenize
from datasets import load_dataset
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding, Dropout, Bidirectional
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import pandas as pd

nltk.download('punkt')

# Load the IMDB dataset
imdb = load_dataset("imdb")

train_dataset = imdb['train'].shuffle(seed=42).select(range(int(len(imdb['train']) * 0.1)))
test_dataset = imdb['test'].shuffle(seed=42).select(range(int(len(imdb['test']) * 0.1)))

y_train = np.array(train_dataset['label'])
y_test = np.array(test_dataset['label'])

# Load pre-trained Word2Vec model
word2vec_model = api.load("word2vec-google-news-300")

max_length = 100

def get_word2vec_sequence(text, model, vector_size):
    words = word_tokenize(text.lower())
    word_vectors = [model[word] for word in words if word in model]
    # Pad or truncate to ensure the same length
    if len(word_vectors) < max_length:
        word_vectors += [[0]*vector_size] * (max_length - len(word_vectors))
    return word_vectors[:max_length]

# Convert texts to sequences of word vectors
X_train_seq = [get_word2vec_sequence(text, word2vec_model, 300) for text in train_dataset['text']]
X_test_seq = [get_word2vec_sequence(text, word2vec_model, 300) for text in test_dataset['text']]

# Convert to numpy arrays
X_train_padded = np.array(X_train_seq, dtype='float32')
X_test_padded = np.array(X_test_seq, dtype='float32')

# Build LSTM model
model = Sequential()
model.add(Bidirectional(LSTM(128, return_sequences=True), input_shape=(max_length, 300)))
model.add(Dropout(0.5))

# Add another LSTM layer
model.add(LSTM(64, return_sequences=False))
model.add(Dropout(0.5))

# Add a dense layer
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.5))

# Output layer
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train_padded, y_train, epochs=5, batch_size=64, validation_split=0.2)

# Predict and evaluate
y_pred_prob = model.predict(X_test_padded)
y_pred = (y_pred_prob > 0.5).astype(int).flatten()

accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
precision = precision_score(y_test, y_pred, average='weighted')
cm = confusion_matrix(y_test, y_pred)

print("Accuracy:", accuracy)
print("F1 Score:", f1)
print("Recall:", recall)
print("Precision:", precision)
print("Confusion Matrix:\n", cm)

# Add results to a DataFrame if needed
new_row = pd.DataFrame({'model': ['Word2Vec + LSTM'], 'F1': [f1], 'recall': [recall], 'precision': [precision]})
results_df = pd.concat([results_df, new_row], ignore_index=True)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 11s 57ms/step - accuracy: 0.5001 - loss: 0.6968 - val_accuracy: 0.5580 - val_loss: 0.6838
Epoch 2/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 4s 31ms/step - accuracy: 0.6111 - loss: 0.6689 - val_accuracy: 0.7080 - val_loss: 0.5705
Epoch 3/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 27ms/step - accuracy: 0.6954 - loss: 0.5918 - val_accuracy: 0.7240 - val_loss: 0.5581
Epoch 4/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 27ms/step - accuracy: 0.7536 - loss: 0.5305 - val_accuracy: 0.7460 - val_loss: 0.5206
Epoch 5/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 36ms/step - accuracy: 0.7709 - loss: 0.5171 - val_accuracy: 0.7180 - val_loss: 0.5406
79/79 ━━━━━━━━━━━━━━━━━━━━ 2s 17ms/step
Accuracy: 0.7416
F1 Score: 0.7387120200554163
Recall: 0.7416
Precision: 0.7537212350108583
Confusion Matrix:
 [[1055  188]
 [ 458  799]]


## TfidfVectorizer + LR

In [8]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from datasets import Dataset, load_dataset
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import numpy as np
import gensim.downloader as api
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score, confusion_matrix
from sklearn.feature_extraction.text import TfidfVectorizer

# Load the IMDB dataset
imdb = load_dataset("imdb")

train_dataset = imdb['train'].shuffle(seed=42).select(range(int(len(imdb['train']) * 0.1)))
test_dataset = imdb['test'].shuffle(seed=42).select(range(int(len(imdb['test']) * 0.1)))

y_train = np.array(train_dataset['label'])
y_test = np.array(test_dataset['label'])

vectorizer = TfidfVectorizer(max_features=5000)
X_train = vectorizer.fit_transform(train_dataset['text'])
X_test = vectorizer.transform(test_dataset['text'])

model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
precision = precision_score(y_test, y_pred, average='weighted')
cm = confusion_matrix(y_test, y_pred)

print("Accuracy:", accuracy)
print("F1 Score:", f1)
print("Recall:", recall)
print("Precision:", precision)
print("Confusion Matrix:\n", cm)

new_row = pd.DataFrame({'model': ['TfidfVectorizer'], 'F1': [f1], 'recall': [recall], 'precision': [precision]})
results_df = pd.concat([results_df, new_row], ignore_index=True)  # If results_df exists

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Accuracy: 0.8336
F1 Score: 0.8333804939301929
Recall: 0.8336
Precision: 0.8350515575990255
Confusion Matrix:
 [[ 993  250]
 [ 166 1091]]


## TFif Vectorizer + naive bayes

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline
from datasets import load_dataset
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score, confusion_matrix
import numpy as np

# Load the IMDB dataset
imdb = load_dataset("imdb")

train_dataset = imdb['train'].shuffle(seed=42).select(range(int(len(imdb['train']) * 0.1)))
test_dataset = imdb['test'].shuffle(seed=42).select(range(int(len(imdb['test']) * 0.1)))

# Extract text and labels
X_train = train_dataset['text']
y_train = np.array(train_dataset['label'])

X_test = test_dataset['text']
y_test = np.array(test_dataset['label'])

# Create a pipeline with Tfidf and Naive Bayes
model = make_pipeline(TfidfVectorizer(max_features=5000), MultinomialNB())

# Train the model
model.fit(X_train, y_train)

# Predict and evaluate
y_pred = model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
precision = precision_score(y_test, y_pred, average='weighted')
cm = confusion_matrix(y_test, y_pred)

print("Accuracy:", accuracy)
print("F1 Score:", f1)
print("Recall:", recall)
print("Precision:", precision)
print("Confusion Matrix:\n", cm)

# Add results to a DataFrame if needed
new_row = pd.DataFrame({'model': ['Naive Bayes'], 'F1': [f1], 'recall': [recall], 'precision': [precision]})
results_df = pd.concat([results_df, new_row], ignore_index=True)  # If results_df exists

Accuracy: 0.8196
F1 Score: 0.8191987209345015
Recall: 0.8196
Precision: 0.8229266691463641
Confusion Matrix:
 [[1080  163]
 [ 288  969]]


## GloVe + LR

In [9]:
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip glove.6B.zip

--2024-10-10 18:32:56--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2024-10-10 18:32:56--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2024-10-10 18:32:56--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glov

In [11]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from datasets import Dataset, load_dataset
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import numpy as np
import gensim.downloader as api
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score, confusion_matrix

# Load the IMDB dataset
imdb = load_dataset("imdb")

train_dataset = imdb['train'].shuffle(seed=42).select(range(int(len(imdb['train']) * 0.1)))
test_dataset = imdb['test'].shuffle(seed=42).select(range(int(len(imdb['test']) * 0.1)))

y_train = np.array(train_dataset['label'])
y_test = np.array(test_dataset['label'])

# Extract the text and labels
train_texts = train_dataset['text']
test_texts = test_dataset['text']

def load_glove_embeddings(glove_file):
    embeddings_index = {}
    with open(glove_file, 'r', encoding='utf-8') as f:
        for line in f:
            values = line.split()
            word = values[0]
            vector = np.asarray(values[1:], dtype='float32')
            embeddings_index[word] = vector
    return embeddings_index

# Load GloVe 100d embeddings
glove_embeddings = load_glove_embeddings('glove.6B.300d.txt')

def get_average_glove_embedding(text, embeddings_index, embedding_dim=100):
    words = word_tokenize(text.lower())  # Tokenize the review
    word_vectors = [embeddings_index[word] for word in words if word in embeddings_index]
    if len(word_vectors) > 0:
        return np.mean(word_vectors, axis=0)  # Average the word vectors
    else:
        return np.zeros(embedding_dim)  # Return a vector of zeros if no words are found

# Convert the train and test reviews into GloVe embeddings
X_train = np.array([get_average_glove_embedding(text, glove_embeddings, embedding_dim=100) for text in train_texts])
X_test = np.array([get_average_glove_embedding(text, glove_embeddings, embedding_dim=100) for text in test_texts])

model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
precision = precision_score(y_test, y_pred, average='weighted')
cm = confusion_matrix(y_test, y_pred)

print("Accuracy:", accuracy)
print("F1 Score:", f1)
print("Recall:", recall)
print("Precision:", precision)
print("Confusion Matrix:\n", cm)

new_row = pd.DataFrame({'model': ['GloVe'], 'F1': [f1], 'recall': [recall], 'precision': [precision]})
results_df = pd.concat([results_df, new_row], ignore_index=True)  # If results_df exists

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Accuracy: 0.7916
F1 Score: 0.7916004334720694
Recall: 0.7916
Precision: 0.7916436776783338
Confusion Matrix:
 [[989 254]
 [267 990]]


## FastText

In [12]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from datasets import Dataset, load_dataset
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import numpy as np
import gensim.downloader as api
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score, confusion_matrix
import fasttext

# Load the IMDB dataset
imdb = load_dataset("imdb")

train_dataset = imdb['train'].shuffle(seed=42).select(range(int(len(imdb['train']) * 0.1)))
test_dataset = imdb['test'].shuffle(seed=42).select(range(int(len(imdb['test']) * 0.1)))

y_train = np.array(train_dataset['label'])
y_test = np.array(test_dataset['label'])

# Extract the text and labels
train_texts = train_dataset['text']
test_texts = test_dataset['text']

# Prepare the text files in the format FastText expects
def save_to_fasttext_format(dataset, file_path):
    with open(file_path, 'w') as f:
        for text, label in zip(dataset['text'], dataset['label']):
            label_prefix = "__label__positive" if label == 1 else "__label__negative"
            f.write(label_prefix + " " + text.replace('\n', ' ') + "\n")

# Save train and test datasets in FastText format
save_to_fasttext_format(train_dataset, 'fasttext_train.txt')
save_to_fasttext_format(test_dataset, 'fasttext_test.txt')


# Train the FastText model
model = fasttext.train_supervised(input="fasttext_train.txt", epoch=25, lr=0.85, wordNgrams=2, verbose=2)

# Save the model
model.save_model("fasttext_imdb_model.bin")

predictions = []
for text in test_texts:
    predicted_label, _ = model.predict(text)
    # Extract the label from the prediction
    predictions.append(predicted_label[0].replace('__label__', ''))

# Convert predictions from labels to binary format (0 or 1)
y_pred = [1 if pred == "positive" else 0 for pred in predictions]

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
precision = precision_score(y_test, y_pred, average='weighted')
cm = confusion_matrix(y_test, y_pred)

print("Accuracy:", accuracy)
print("F1 Score:", f1)
print("Recall:", recall)
print("Precision:", precision)
print("Confusion Matrix:\n", cm)

new_row = pd.DataFrame({'model': ['FastText'], 'F1': [f1], 'recall': [recall], 'precision': [precision]})
results_df = pd.concat([results_df, new_row], ignore_index=True)  # If results_df exists

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Accuracy: 0.8136
F1 Score: 0.8134975726442303
Recall: 0.8136
Precision: 0.8141003946712645
Confusion Matrix:
 [[ 984  259]
 [ 207 1050]]


## Results

In [13]:
results_df.T # Best F1 scors are 'CountVectorizer with LogisticRegression' and 'TfldfVectorizer with LogisticRegression'

,0,1,2,3,4,5,6,7
model,BERT,CountVectorizer,Word2Vec,Word2Vec + LSTM,TfidfVectorizer,Naive Bayes,GloVe,FastText
F1,0.799135,0.836612,0.798,0.738712,0.83338,0.819199,0.7916,0.813498
recall,0.806,0.8368,0.798,0.7416,0.8336,0.8196,0.7916,0.8136
precision,0.8525,0.83806,0.797999,0.753721,0.835052,0.822927,0.791644,0.8141
